In [12]:
# BLOCK 1: Load All Files and Print Columns Neatly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tabulate import tabulate
import os

# Set paths
merged_log_path = r'E:\FYP\FYP Symposium\Merged Log (49).xlsx'
stock_data_path = r'E:\FYP\FYP Symposium\Trading Simulation Experiment Data Turn Wise.xlsx'
dta_path = r'E:\FYP\FYP Symposium\SurveysClean.dta'

# Load Data
merged_log = pd.read_excel(merged_log_path)
stock_xls = pd.ExcelFile(stock_data_path)
stock_tables = {stock: pd.read_excel(stock_xls, sheet_name=stock) for stock in ['TSLA', 'XOM', 'NFLX', 'PG']}
strategy = pd.read_stata(dta_path)

# Rename columns in each stock sheet
rename_map = {
    'trend': 'Close_price_diff',
    'trend direction': 'price_trend_1',
    'trend summary': 'price_trend_7',
    'volume trend change': 'volume_diff',
    'volume trend direction': 'volume_trend_1',
    'volume trend summary': 'volume_trend_7',
    'Technical Decision': 'MACD_trend',
    'Bollinger Classification': 'bollinger_trend'
}

for stock_name, df in stock_tables.items():
    stock_tables[stock_name].rename(columns=rename_map, inplace=True)


# Show loaded columns for verification (pretty print using tabulate)
print("Merged Log Columns:\n")
print(tabulate([[col] for col in merged_log.columns], headers=["Merged Log Columns"], tablefmt="github"))

print("\nStrategy Columns:\n")
print(tabulate([[col] for col in strategy.columns], headers=["Strategy Columns"], tablefmt="github"))

print(merged_log.head(15))

Merged Log Columns:

| Merged Log Columns    |
|-----------------------|
| Date                  |
| Real Time             |
| Simulation Time       |
| Seconds left          |
| Participant_ID        |
| turn                  |
| action                |
| ticker                |
| News Sentiment        |
| News Truth            |
| quantity              |
| Stock Price           |
| Total Stock           |
| cash_before           |
| cash_after            |
| stockportfolio_before |
| stockportfolio_after  |
| Total_assets          |
| total_TSLA_holding    |
| total_XOM_holding     |
| total_NFLX_holding    |
| total_PG_holding      |
| TSLA_value            |
| XOM_value             |
| NFLX_value            |
| PG_value              |

Strategy Columns:

| Strategy Columns   |
|--------------------|
| TimePre            |
| Name               |
| Participant_ID     |
| PRM1_5             |
| PRM1_6             |
| PRM1_7             |
| PRM1_8             |
| PRM1_9             |
|

In [13]:
# BLOCK 2: Rename Columns for Consistency

# Rename Merged Log Columns to match our expectations
merged_log.rename(columns={
    'News Sentiment': 'news_sentiment',
    'News Truth': 'news_truth'
}, inplace=True)

# Rename Strategy Columns if needed
if 'participant_id' not in strategy.columns:
    if 'ResponseID' in strategy.columns:
        strategy.rename(columns={'ResponseID': 'participant_id'}, inplace=True)
    elif 'ResponseId' in strategy.columns:
        strategy.rename(columns={'ResponseId': 'participant_id'}, inplace=True)

# Check after renaming
print("Merged Log Columns (After Rename):\n")
print(tabulate([[col] for col in merged_log.columns], headers=["Merged Log Columns"], tablefmt="github"))

print("\nStrategy Columns (After Rename):\n")
print(tabulate([[col] for col in strategy.columns], headers=["Strategy Columns"], tablefmt="github"))

import pandas as pd
from tabulate import tabulate

# Assuming merged_log and strategy are already available after renaming

# Export both merged_log and strategy to Excel with separate sheets
output_file = r'E:\FYP\FYP Symposium\Outputs\Renamed_Merged_Log_and_Strategy.xlsx'  # Specify the file path

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    merged_log.to_excel(writer, sheet_name='Merged_Log', index=False)  # Save merged_log to the first sheet
    strategy.to_excel(writer, sheet_name='Strategy', index=False)  # Save strategy to the second sheet

print(f"The renamed merged_log and strategy have been exported to: {output_file}")
strategy.head(15)

# Define scoring function
def assign_score(row, method_name):
    score = 0
    for i, weight in zip(['1st', '2nd', '3rd'], [5, 3.5, 2]):
        factor = row.get(f'DecisionFactor_{i}', None)
        if factor == method_name:
            score = weight
            break
    return score

# Add scoring for 'Graph'
strategy['Graph Scoring'] = strategy.apply(lambda row: assign_score(row, 'Graph'), axis=1)

# Add scoring for 'Data Table'
strategy['Data Table Scoring'] = strategy.apply(lambda row: assign_score(row, 'Data Table'), axis=1)

# Add column for average of Graph Scoring and Data Table Scoring
strategy['Average Scoring'] = strategy[['Graph Scoring', 'Data Table Scoring']].mean(axis=1)

# View result
print(strategy[['Participant_ID', 'DecisionFactor_1st', 'DecisionFactor_2nd', 'DecisionFactor_3rd',
                'Graph Scoring', 'Data Table Scoring', 'Average Scoring']].head(10))

# Final export with both sheets: updated strategy and merged_log
final_output_file = r'E:\FYP\FYP Symposium\Outputs\Final_Merged_Log_and_Strategy_With_Scoring.xlsx'

with pd.ExcelWriter(final_output_file, engine='xlsxwriter') as writer:
    merged_log.to_excel(writer, sheet_name='Merged_Log', index=False)
    strategy.to_excel(writer, sheet_name='Strategy', index=False)

print(f"✅ Both updated sheets exported to: {final_output_file}")

print("The updated strategy with scoring has been exported to: E:\\FYP\\FYP Symposium\\Outputs\\Updated_Strategy_With_Scoring.xlsx")

Merged Log Columns (After Rename):

| Merged Log Columns    |
|-----------------------|
| Date                  |
| Real Time             |
| Simulation Time       |
| Seconds left          |
| Participant_ID        |
| turn                  |
| action                |
| ticker                |
| news_sentiment        |
| news_truth            |
| quantity              |
| Stock Price           |
| Total Stock           |
| cash_before           |
| cash_after            |
| stockportfolio_before |
| stockportfolio_after  |
| Total_assets          |
| total_TSLA_holding    |
| total_XOM_holding     |
| total_NFLX_holding    |
| total_PG_holding      |
| TSLA_value            |
| XOM_value             |
| NFLX_value            |
| PG_value              |

Strategy Columns (After Rename):

| Strategy Columns   |
|--------------------|
| TimePre            |
| Name               |
| Participant_ID     |
| PRM1_5             |
| PRM1_6             |
| PRM1_7             |
| PRM1_8         

In [14]:
# BLOCK 3: Extract PRM2b_14a indicator knowledge cleanly

def check_knowledge(prm2b_14a_response):
    if pd.isna(prm2b_14a_response):
        return {'MACD': False, 'Bollinger': False, 'Volume': False, 'HighLow_OpenClose': False}

    text = prm2b_14a_response.lower()
    
    knowledge = {
        'MACD': False,
        'Bollinger': False,
        'Volume': False,
        'HighLow_OpenClose': False
    }

    # Check MACD knowledge
    if 'macd' in text:
        knowledge['MACD'] = True
    
    # Check Bollinger Bands
    if 'bollinger' in text or 'bb' in text:
        knowledge['Bollinger'] = True

    # Check Volume
    if 'volume' in text:
        knowledge['Volume'] = True

    # Check High Low Open Close
    if 'high' in text or 'low' in text or 'open' in text or 'close' in text or 'ohlc' in text:
        knowledge['HighLow_OpenClose'] = True

    return knowledge

# --- Apply to all participants ---

knowledge_records = []

for idx, row in strategy.iterrows():
    participant_id = row['Participant_ID']
    prm2b_14a = row.get('PRM2b_14a', None)
    knowledge = check_knowledge(prm2b_14a)

    knowledge_records.append({
        'Participant_ID': participant_id,
        'Knows_MACD': 'Yes' if knowledge['MACD'] else 'No',
        'Knows_Bollinger': 'Yes' if knowledge['Bollinger'] else 'No',
        'Knows_Volume': 'Yes' if knowledge['Volume'] else 'No',
        'Knows_HighLow_OpenClose': 'Yes' if knowledge['HighLow_OpenClose'] else 'No'
    })

# Create DataFrame
knowledge_df = pd.DataFrame(knowledge_records)

# Drop 'Knows_HighLow_OpenClose' — no longer needed
if 'Knows_HighLow_OpenClose' in knowledge_df.columns:
    knowledge_df.drop(columns=['Knows_HighLow_OpenClose'], inplace=True)

# --- Merge Cleaned Knowledge into Strategy ---

# Drop old versions to avoid MergeError
strategy.drop(columns=['Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knows_HighLow_OpenClose', 'Knowledge_Score'], errors='ignore', inplace=True)

# Merge new knowledge
strategy = pd.merge(strategy, knowledge_df, on='Participant_ID', how='left')
print("\n✅ Merged updated knowledge into strategy successfully.")

# --- Scoring Logic ---

# Map Yes/No with new scoring logic (Volume: Yes = 1, No = 0.5)
knowledge_df_numeric = knowledge_df.copy()
knowledge_df_numeric['Knows_MACD'] = knowledge_df_numeric['Knows_MACD'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Bollinger'] = knowledge_df_numeric['Knows_Bollinger'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Volume'] = knowledge_df_numeric['Knows_Volume'].map({'Yes': 1, 'No': 0.5})

# Compute score
knowledge_df['Knowledge_Score'] = (
    knowledge_df_numeric['Knows_MACD'] +
    knowledge_df_numeric['Knows_Bollinger'] +
    knowledge_df_numeric['Knows_Volume']
)

# Show sample output
print("\n✅ Final Knowledge Score (range 0.5–3.0):")
display(knowledge_df[['Participant_ID', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knowledge_Score']].head())

# Optional documentation string
knowledge_score_note = "Knowledge_Score = Knows_MACD (1/0) + Knows_Bollinger (1/0) + Knows_Volume (1 if Yes, 0.5 if No); Range = 0.5 to 3.0"

# Export to Excel
output_knowledge_path = r'E:\FYP\FYP Symposium\Outputs\Knowledge_Score_Updated.xlsx'
knowledge_df.to_excel(output_knowledge_path, index=False)
print(f"\n✅ Updated Knowledge Data (with Knowledge_Score) exported to: {output_knowledge_path}")



✅ Merged updated knowledge into strategy successfully.

✅ Final Knowledge Score (range 0.5–3.0):


,Participant_ID,Knows_MACD,Knows_Bollinger,Knows_Volume,Knowledge_Score
0,E0070,No,No,No,0.5
1,E0169,Yes,Yes,Yes,3.0
2,E0426,No,No,No,0.5
3,E0712,No,No,No,0.5
4,E1130,Yes,No,No,1.5



✅ Updated Knowledge Data (with Knowledge_Score) exported to: E:\FYP\FYP Symposium\Outputs\Knowledge_Score_Updated.xlsx


In [15]:
# ========== BLOCK 4: Factual trends (MACD/Bollinger/Price/Volume signals) ==========
import pandas as pd
import os

# --- 3.1: Check if participant knows indicator
def participant_knows_indicator(prm2b_14a_response, indicator):
    if pd.isnull(prm2b_14a_response):
        return False
    return indicator.lower() in prm2b_14a_response.lower()

# --- 3.2: Detect market signals from price data
def detect_market_signals(stock_df, turn, days=5):
    visible_data = stock_df[stock_df['Turn'] <= turn].sort_values(by='Turn', ascending=False).head(days)

    price_trend = 'neutral'
    volume_spike = False
    macd_signal = 'neutral'
    bb_trend = 'neutral'

    if len(visible_data) >= 3:
        close_now = visible_data.iloc[0]['Close']
        close_past = visible_data.iloc[-1]['Close']
        change_pct = (close_now - close_past) / close_past * 100

        if change_pct > 2.0:
            price_trend = 'uptrend'
        elif change_pct < -2.0:
            price_trend = 'downtrend'
        else:
            price_trend = 'neutral'

        avg_volume = visible_data['Volume'].mean()
        curr_volume = visible_data.iloc[0]['Volume']
        if curr_volume > 1.2 * avg_volume:
            volume_spike = True

        macd = visible_data.iloc[0]['MACD (12,26,9)']
        signal = visible_data.iloc[0]['Signal (12,26,9)']
        macd_hist = visible_data.iloc[0]['MACD Histogram (12,26,9)']

        if macd > signal and macd_hist > 0:
            macd_signal = 'buy'
        elif macd < signal and macd_hist < 0:
            macd_signal = 'sell'

        close = visible_data.iloc[0]['Close']
        top_bb = visible_data.iloc[0]['Top Bollinger Bands (20,O,2,ma,n)']
        bottom_bb = visible_data.iloc[0]['Bottom Bollinger Bands (20,O,2,ma,n)']
        if close > top_bb:
            bb_trend = 'overbought'
        elif close < bottom_bb:
            bb_trend = 'oversold'
        else:
            bb_trend = 'neutral'

    return price_trend, volume_spike, macd_signal, bb_trend

# --- Suggest Buy Logic
def suggest_buy_decision(price_trend, macd_signal, bb_trend, volume_spike, volatility):
    score = 0
    if price_trend == 'uptrend': score += 1
    if macd_signal == 'buy': score += 1
    if bb_trend == 'oversold': score += 1
    if volume_spike: score += 1
    if volatility < 10: score += 1
    return 'Attractive' if score >= 2 else 'Risky'

# --- Load raw stock data
experiment_file_path = r'E:\FYP\FYP Symposium\Trading Simulation Experiment Data Turn Wise.xlsx'
stock_tables = {}

for stock in ['TSLA', 'XOM', 'NFLX', 'PG']:
    df = pd.read_excel(experiment_file_path, sheet_name=stock)
    df = df.sort_values(by='Turn', ascending=False).reset_index(drop=True)
    stock_tables[stock] = df

# ========== TURN-WISE TREND METRICS CREATION ==========
file_path = r'E:\FYP\FYP Symposium\Turn Data in descending order.xlsx'
sheet_names = ['TSLA', 'NFLX', 'PG', 'XOM']
turn_data = {}

for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)

    # Price trend
    df['trend change'] = df['Close'] - df['Close'].shift(-1)

    def get_trend_direction(change):
        if pd.isna(change): return None
        elif change > 0: return 'uptrend'
        elif change < 0: return 'downtrend'
        else: return 'neutral'

    df['price_trend_1'] = df['trend change'].apply(get_trend_direction)

    def summarize_trend_block(group):
        up = (group['price_trend_1'] == 'uptrend').sum()
        down = (group['price_trend_1'] == 'downtrend').sum()
        group['price_trend_7'] = 'uptrend' if up > down else 'downtrend' if down > up else 'equal'
        return group

    df = df.groupby('Turn', group_keys=False).apply(summarize_trend_block)

    # Volume trend
    df['volume_diff'] = df['Volume'] - df['Volume'].shift(-1)

    def get_volume_trend_direction(change):
        if pd.isna(change): return None
        elif change > 0: return 'uptrend'
        elif change < 0: return 'downtrend'
        else: return 'neutral'

    df['volume_trend_1'] = df['volume_diff'].apply(get_volume_trend_direction)

    def summarize_volume_trend_block(group):
        up = (group['volume_trend_1'] == 'uptrend').sum()
        down = (group['volume_trend_1'] == 'downtrend').sum()
        group['volume_trend_7'] = 'uptrend' if up > down else 'downtrend' if down > up else 'equal'
        return group

    df = df.groupby('Turn', group_keys=False).apply(summarize_volume_trend_block)

    # MACD
    def make_technical_decision(row):
        try:
            macd = row['MACD (12,26,9)']
            signal = row['Signal (12,26,9)']
            hist = row['MACD Histogram (12,26,9)']
            if macd > signal > hist: return 'Buy'
            elif macd < signal and macd < hist: return 'Sell'
            else: return 'Neutral'
        except: return 'Neutral'

    df['MACD_trend'] = df.apply(make_technical_decision, axis=1)

    # Bollinger
    def classify_bollinger(row):
        try:
            if row['Close'] > row['Top Bollinger Bands (20,O,2,ma,n)']:
                return 'Over Bought'
            elif row['Close'] < row['Bottom Bollinger Bands (20,O,2,ma,n)']:
                return 'Over Sold'
            else:
                return 'Neutral'
        except:
            return 'Neutral'

    df['bollinger_trend'] = df.apply(classify_bollinger, axis=1)

    df['MACD_trend_7'] = df.groupby('Turn')['MACD_trend'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    df['bollinger_trend_7'] = df.groupby('Turn')['bollinger_trend'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

    turn_data[sheet] = df

# === Export full trend summary ===
output_path = r'E:\FYP\FYP Symposium\Outputs\Combined_Turn_With_Trend_Summary.xlsx'
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for stock, df in turn_data.items():
        df.to_excel(writer, sheet_name=stock, index=False)

print(f"✅ Exported successfully with trend summaries (Close & Volume) for all sheets to: {output_path}")

# === NEW: Load only Turn 1–5 top rows for enrichment ===
turn_summary_1to6 = {}
for stock in ['TSLA', 'NFLX', 'PG', 'XOM']:
    df = pd.read_excel(output_path, sheet_name=stock)
    top_rows = df.drop_duplicates(subset='Turn', keep='first')
    top_rows = top_rows[top_rows['Turn'].isin([6, 5, 4, 3, 2, 1])]
    top_rows = top_rows.sort_values(by='Turn', ascending=False)
    turn_summary_1to6[stock] = top_rows
    
# === Precompute volatility per Turn per stock from raw Turn Data ===
volatility_lookup = {}
for stock in ['TSLA', 'NFLX', 'PG', 'XOM']:
    df = pd.read_excel(r'E:\FYP\FYP Symposium\Turn Data in descending order.xlsx', sheet_name=stock)
    vol_by_turn = df.groupby('Turn')['Close'].std().reset_index()
    vol_by_turn.rename(columns={'Close': 'Turn_Volatility'}, inplace=True)
    volatility_lookup[stock] = vol_by_turn

# === Generate trend data table and file ===
def generate_trend_basis(stock_df, stock_name):
    trend_rows = []
    def map_score(val, pos_list, neg_list):
                val = str(val).lower()
                if val in pos_list:
                    return 1
                elif val in neg_list:
                    return -1
                return 0
    for turn in sorted(stock_df['Turn'].unique()):
        visible_data = stock_df[stock_df['Turn'] <= turn].sort_values(by='Turn', ascending=False).head(5)
        if visible_data.empty:
            continue
        try:
            price_trend, volume_spike, macd_signal, bb_trend = detect_market_signals(stock_df, turn)
            close = visible_data.iloc[0]['Close']
            high = visible_data.iloc[0]['High']
            low = visible_data.iloc[0]['Low']
            vol_df = volatility_lookup[stock_name]
            vol_row = vol_df[vol_df['Turn'] == turn]
            volatility = vol_row['Turn_Volatility'].values[0] if not vol_row.empty else None
            macd_hist = visible_data.iloc[0]['MACD Histogram (12,26,9)']
            macd_hist_strength = round(macd_hist, 4)
            volume_support = volume_spike and turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'volume_trend_7'] == 'uptrend'
            trend_reversal = False
            turn_list = stock_df['Turn'].sort_values(ascending=False).unique()
            turn_idx = list(turn_list).index(turn)
            if turn_idx + 1 < len(turn_list):
                next_turn = turn_list[turn_idx + 1]
                df_enriched = turn_summary_1to6.get(stock_name)
                if df_enriched is not None and turn in df_enriched['Turn'].values and next_turn in df_enriched['Turn'].values:
                    current_trend = df_enriched.set_index('Turn').loc[turn, 'price_trend_7']
                    next_trend = df_enriched.set_index('Turn').loc[next_turn, 'price_trend_7']
                    trend_reversal = current_trend != next_trend
            
            # --- Scoring each indicator (individual columns) ---
            score_price_trend = map_score(price_trend, ['uptrend'], ['downtrend'])
            score_macd_signal = map_score(macd_signal, ['buy'], ['sell'])
            score_bb_trend = map_score(bb_trend, ['oversold'], ['overbought'])
            score_volume_spike = 1 if volume_spike else 0

            # --- Summary trend scores ---
            summary_df = turn_summary_1to6[stock_name].set_index('Turn')
            if turn not in summary_df.index:
                print(f"⚠️ Turn {turn} not found in turn_summary for {stock_name}")
                continue
            summary = summary_df.loc[turn]

            score_volume_trend = map_score(summary['volume_trend_7'], ['uptrend'], ['downtrend'])
            score_macd_trend = map_score(summary['MACD_trend_7'], ['buy'], ['sell'])
            score_bb_summary = map_score(summary['bollinger_trend_7'], ['oversold'], ['overbought'])
            score_price_summary = map_score(summary['price_trend_7'], ['uptrend'], ['downtrend'])
            score_trend_reversal = map_score('Yes' if trend_reversal else 'No', ['Yes'], [])
            score_volume_support = map_score('Yes' if volume_support else 'No', ['Yes'], [])

            # --- Final score calculation ---
            
            score = (
                score_price_trend +
                score_macd_signal +
                score_bb_trend +
                score_volume_spike +
                score_volume_trend +
                score_macd_trend +
                score_bb_summary +
                score_price_summary +
                score_trend_reversal +
                score_volume_support
            )


            score = round(score, 2)  # Ensure consistent float precision
            
            # Summary trend indicators
            score += (
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'volume_trend_7'], ['uptrend'], ['downtrend']) +
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'MACD_trend_7'], ['buy'], ['sell']) +
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'bollinger_trend_7'], ['oversold'], ['overbought']) +
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'price_trend_7'], ['uptrend'], ['downtrend']) +
                map_score('Yes' if trend_reversal else 'No', ['Yes'], []) +
                map_score('Yes' if volume_support else 'No', ['Yes'], [])
            )
            # --- Final buy suggestion logic ---
            if score >= 3.0:
                buy_suggestion = 'Attractive'
            elif score >= 1.0:
                buy_suggestion = 'Cautious'
            else:
                buy_suggestion = 'Risky'

            

            trend_rows.append({
                'Turn': turn,
                'Close Price': close,
                'High Price': high,
                'Low Price': low,
                'Volatility': volatility,

                # Raw factual indicators
                'Price Trend': price_trend,
                'Score - Price Trend': score_price_trend,
                'MACD Signal': macd_signal,
                'Score - MACD Signal': score_macd_signal,
                'Bollinger Band Trend': bb_trend,
                'Score - Bollinger Band': score_bb_trend,
                'Volume Spike': volume_spike,
                'Score - Volume Spike': score_volume_spike,
                'MACD Histogram': macd_hist,
                'MACD Histogram Strength': macd_hist_strength,

                # Summary trends
                'price_trend_7': summary['price_trend_7'],
                'Score - Price Trend (7)': score_price_summary,
                'volume_trend_7': summary['volume_trend_7'],
                'Score - Volume Trend (7)': score_volume_trend,
                'MACD_trend_7': summary['MACD_trend_7'],
                'Score - MACD Trend (7)': score_macd_trend,
                'bollinger_trend_7': summary['bollinger_trend_7'],
                'Score - Bollinger Trend (7)': score_bb_summary,

                # Flags
                'Trend Reversal Flag': 'Yes' if trend_reversal else 'No',
                'Score - Trend Reversal': score_trend_reversal,
                'Volume Trend Support': 'Yes' if volume_support else 'No',
                'Score - Volume Support': score_volume_support,

                # Final decision
                'Buy Confidence Score': round(score, 2),
                'Is Attractive to Buy?': buy_suggestion
            })


        except Exception as e:
            print(f"⚠️ Error processing Turn {turn} in {stock_name}: {e}")
            continue

    trend_df = pd.DataFrame(trend_rows)
    
    # ✅ Normalize Buy Confidence Score to 1–5 after DataFrame is built
    if 'Buy Confidence Score' in trend_df.columns:
        min_score = trend_df['Buy Confidence Score'].min()
        max_score = trend_df['Buy Confidence Score'].max()
        if min_score != max_score:
            trend_df['Buy Confidence Score_Normalized'] = (
                (trend_df['Buy Confidence Score'] - min_score) / (max_score - min_score) * 4 + 1
            ).round(2)
        else:
            trend_df['Buy Confidence Score_Normalized'] = 3

    # === Combined Signal-Based Trend Reversal ===
    trend_df['Trend Reversal Flag'] = trend_df['price_trend_7'] != trend_df['price_trend_7'].shift(-1)
    trend_df['Trend Reversal Flag'] = trend_df['Trend Reversal Flag'].apply(lambda x: 'Yes' if x else 'No')

    trend_df['Volume Trend Support'] = trend_df.apply(
        lambda row: 'Yes' if row['Volume Spike'] and row['volume_trend_7'] == 'uptrend' else 'No',
        axis=1
    )

    # === Combined Signal-Based Trend Reversal ===
    trend_df['Trend Reversal Flag'] = trend_df['price_trend_7'] != trend_df['price_trend_7'].shift(-1)
    trend_df['Trend Reversal Flag'] = trend_df['Trend Reversal Flag'].apply(lambda x: 'Yes' if x else 'No')

    trend_df['Volume Trend Support'] = trend_df.apply(
        lambda row: 'Yes' if row['Volume Spike'] and row['volume_trend_7'] == 'uptrend' else 'No',
        axis=1
    )


    # === Strict grouped column ordering ===
    ordered_cols = [

        # Basic Info
        'Turn', 'Close Price', 'High Price', 'Low Price', 'Volatility',

        # PRICE-RELATED
        'Price Trend', 'Score - Price Trend',
        'price_trend_7', 'Score - Price Trend (7)',
        'Trend Reversal Flag', 'Score - Trend Reversal',

        # MACD-RELATED
        'MACD Signal', 'Score - MACD Signal',
        'MACD Histogram', 'MACD Histogram Strength',
        'MACD_trend_7', 'Score - MACD Trend (7)',

        # BOLLINGER-RELATED
        'Bollinger Band Trend', 'Score - Bollinger Band',
        'bollinger_trend_7', 'Score - Bollinger Trend (7)',

        # VOLUME-RELATED
        'Volume Spike', 'Score - Volume Spike',
        'volume_trend_7', 'Score - Volume Trend (7)',
        'Volume Trend Support', 'Score - Volume Support',

        # FINAL OUTPUT
        'Buy Confidence Score', 'Buy Confidence Score_Normalized',
        'Is Attractive to Buy?'
    ]

    # Safely reorder
    trend_df = trend_df[[col for col in ordered_cols if col in trend_df.columns]]



    return trend_df


    
# === Run and display ===
trend_tables = {}
for stock in stock_tables.keys():
    df = generate_trend_basis(stock_tables[stock], stock)
    df = df[df['Turn'] != 6]  # drop Turn 6
    trend_tables[stock] = df  # <-- this line was missing

# Display sample tables without color formatting
for stock, table in trend_tables.items():
    print(f"\nSample Trend Table for {stock}:")
    print(table.head(10))  # Show first 10 rows; adjust as needed
    
# Load Strategy sheet once at the top of BLOCK 5
strategy_path = r"E:\FYP\FYP Symposium\Outputs\Final_Merged_Log_and_Strategy_With_Scoring.xlsx"
strategy_df = pd.read_excel(strategy_path, sheet_name='Strategy')

# Create lookup dictionary for Average Scoring
avg_score_dict = strategy_df.set_index('Participant_ID')['Average Scoring'].to_dict()


# ========== BLOCK 5: Enrich merged logs with trend-based market context ==========

# Load merged log (⚠️ it does NOT contain 'Average Scoring')
merged_log_path = r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx"
merged_log_df = pd.read_excel(merged_log_path, sheet_name='Merged_Log')

# Load Strategy sheet to get Average Scoring
strategy_path = r"E:\FYP\FYP Symposium\Outputs\Final_Merged_Log_and_Strategy_With_Scoring.xlsx"
strategy_df = pd.read_excel(strategy_path, sheet_name='Strategy')
avg_scores = strategy_df.set_index('Participant_ID')['Average Scoring'].to_dict()

# Enrichment function
def enrich_log_with_trends(merged_log_df, trend_tables):
    enriched_rows = []

    for idx, row in merged_log_df.iterrows():
        pid = row['Participant_ID']
        ticker = row['ticker']
        turn = row['turn']

        try:
            trend_row_df = trend_tables.get(ticker)
            if trend_row_df is None:
                print(f"⚠️ No trend data for ticker: {ticker}")
                continue

            trend_row = trend_row_df[trend_row_df['Turn'] == turn]
            if trend_row.empty:
                print(f"⚠️ Turn {turn} not found in trend table for {ticker}")
                continue

            trend_data = trend_row.iloc[0][['Is Attractive to Buy?']].to_dict()
            enriched_row = row.to_dict()
            enriched_row.update(trend_data)
            enriched_rows.append(enriched_row)

        except Exception as e:
            print(f"⚠️ Error processing row {idx}: {e}")
            continue

    return pd.DataFrame(enriched_rows)

# Run enrichment
enriched_log_df = enrich_log_with_trends(merged_log_df, trend_tables)

def compute_action_signal_value(row):
    action = str(row['action']).strip().lower()
    signal = str(row['Is Attractive to Buy?']).strip().lower()
    avg_score = row.get('Average Scoring', 0)

    if action == 'buy' and signal == 'risky':
        return 0
    elif action == 'sell' and signal == 'risky':
        return avg_score
    elif action == 'buy' and signal == 'cautious':
        return avg_score / 2
    elif action == 'sell' and signal == 'cautious':
        return avg_score / 2
    elif action == 'buy' and signal == 'attractive':
        return avg_score
    elif action == 'sell' and signal == 'attractive':
        return 0
    return 0  # Default fallback for unknown values




# ========== BLOCK 7: Add Signal Alignment Score ==========
def compute_alignment_score(row):
    action = str(row['action']).strip().lower()
    signal = str(row['Is Attractive to Buy?']).strip().lower()

    if signal == 'risky':
        return 1 if action == 'sell' else 0
    elif signal == 'cautious':
        return 0.5
    elif signal == 'attractive':
        return 1 if action == 'buy' else 0
    return 0

# Apply the alignment score
enriched_log_df['Signal Alignment Score'] = enriched_log_df.apply(compute_alignment_score, axis=1)

# Add explanation column for Signal Alignment Score
def explain_alignment_score(row):
    pid = row['Participant_ID']
    score = row['Signal Alignment Score']
    avg = avg_scores.get(pid, 0)

    if score == 0:
        return "Misaligned – Action contradicts market signal"
    elif score == 0.5:
        return "Partially Aligned – Somewhat matches the signal"
    elif score == 1:
        return "Fully Aligned – Matches the signal strength"
    
enriched_log_df['Signal Alignment Explanation'] = enriched_log_df.apply(explain_alignment_score, axis=1)

# # Compute overall average alignment per participant
# overall_alignment = (
#     enriched_log_df.groupby('Participant_ID')['Signal Alignment Score']
#     .mean()
#     .round(2)
#     .to_dict()
# )

# # Map it back to each row
# enriched_log_df['Avg Signal Alignment (Overall)'] = enriched_log_df['Participant_ID'].map(overall_alignment)

# # Explain overall alignment quality
# def explain_overall_alignment(score):
#     if score == 0:
#         return "Completely Misaligned – consistently acted against signals"
#     elif score < 2.0:
#         return "Poor Alignment – often disregarded market trends"
#     elif score < 3.5:
#         return "Moderate Alignment – partial awareness of signals"
#     elif score < 4.5:
#         return "Strong Alignment – usually followed signals correctly"
#     else:
#         return "Excellent Alignment – highly in sync with market trends"

# enriched_log_df['Overall Alignment Explanation'] = enriched_log_df['Avg Signal Alignment (Overall)'].apply(explain_overall_alignment)

def infer_actual_usage(row, threshold=3.5):
    pid = row['Participant_ID']
    avg_score = avg_score_dict.get(pid, 0)
    signal_available = str(row.get('Signal Was Available', '')).lower() == 'yes'
    aligned = row.get('Signal Alignment Score', 0) >= 0.5

    claimed_use = avg_score >= threshold

    if signal_available and claimed_use and aligned:
        return "✅ Used Signal – Claimed & Aligned"
    elif signal_available and claimed_use and not aligned:
        return "❌ Ignored Signal – Claimed but not aligned"
    elif signal_available and not claimed_use and aligned:
        return "⚠️ Aligned Accidentally – Didn't claim use"
    elif not signal_available and claimed_use:
        return "⚪ No Signal – Can't assess usage"
    else:
        return "❌ No Use – No claim and no signal used"



def compute_action_signal_value(row):
    pid = row['Participant_ID']
    action = str(row['action']).strip().lower()
    signal = str(row['Is Attractive to Buy?']).strip().lower()
    avg_score = avg_score_dict.get(pid, 0)  # Safe dictionary lookup

    if action == 'buy' and signal == 'risky':
        return 0
    elif action == 'sell' and signal == 'risky':
        return avg_score
    elif action == 'buy' and signal == 'cautious':
        return avg_score / 2
    elif action == 'sell' and signal == 'cautious':
        return avg_score / 2
    elif action == 'buy' and signal == 'attractive':
        return avg_score
    elif action == 'sell' and signal == 'attractive':
        return 0
    return 0  # Default fallback for unknown values

# Apply it
enriched_log_df['Action Signal Value'] = enriched_log_df.apply(compute_action_signal_value, axis=1)

# Save enriched merged log with signal score
final_export_path = r"E:\FYP\FYP Symposium\Outputs\Final_Enriched_Log_With_Signal_Score.xlsx"
enriched_log_df.to_excel(final_export_path, index=False)
print(f"✅ Final enriched log WITH signal alignment score exported to: {final_export_path}")

# ========== Export trend tables ==========

export_path = r'E:\FYP\FYP Symposium\Outputs\Final_Trend_Tables.xlsx'
with pd.ExcelWriter(export_path, engine='openpyxl') as writer:
    for ticker, df in trend_tables.items():
        if not df.empty:
            df.to_excel(writer, sheet_name=ticker, index=False)

print(f"✅ Final enriched trend tables exported to: {export_path}")


C:\Users\MM COMPUTERS\AppData\Local\Temp\ipykernel_26092\3183001729.py:102: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Turn', group_keys=False).apply(summarize_trend_block)
C:\Users\MM COMPUTERS\AppData\Local\Temp\ipykernel_26092\3183001729.py:121: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Turn', group_keys=False).apply(summarize_volume_trend_block)
C:\Users\MM COMPU

✅ Exported successfully with trend summaries (Close & Volume) for all sheets to: E:\FYP\FYP Symposium\Outputs\Combined_Turn_With_Trend_Summary.xlsx

Sample Trend Table for TSLA:
   Turn  Close Price  High Price  Low Price  Volatility Price Trend  \
0   1.0       337.80      343.99     326.20   26.239602     neutral   
1   2.0       372.00      377.59     354.00   11.078977     neutral   
2   3.0       336.34      340.55     316.83   17.719387     neutral   
3   4.0       325.33      326.25     309.42   14.211373   downtrend   
4   5.0       364.65      372.33     356.91   25.304652     uptrend   

   Score - Price Trend price_trend_7  Score - Price Trend (7)  \
0                    0     downtrend                       -1   
1                    0       uptrend                        1   
2                    0     downtrend                       -1   
3                   -1       uptrend                        1   
4                    1       uptrend                        1   

  Tr

In [16]:
#==== Block 8: Output 2 logic different logic but same inferences ====

import pandas as pd
import os

# Define file paths
base_path = r"E:\FYP\FYP Symposium"
output_path = os.path.join(base_path, "Output 2", "Participant_Indicator_Segments.xlsx")

# Load data
log_path = os.path.join(base_path, "Renamed_Merged_Log_and_Strategy.xlsx")
survey_path = os.path.join(base_path, "Post-Survey (Responses).xlsx")

merged_df = pd.read_excel(log_path)
post_df = pd.read_excel(survey_path)

# Extract and rename indicator preferences
rank_cols = [col for col in post_df.columns if any(k in col.lower() for k in ['graph', 'data table', 'macd', 'bollinger'])]
preference_df = post_df[['Participant ID'] + rank_cols].copy()

rename_map = {
    col: 'Graph_Rank' for col in preference_df.columns if 'graph' in col.lower()
}
rename_map.update({
    col: 'DataTable_Rank' for col in preference_df.columns if 'data table' in col.lower()
})
rename_map.update({
    col: 'MACD_Rank' for col in preference_df.columns if 'macd' in col.lower()
})
rename_map.update({
    col: 'BB_Rank' for col in preference_df.columns if 'bollinger' in col.lower()
})
preference_df.rename(columns=rename_map, inplace=True)

# Compute trend signals
merged_df_sorted = merged_df.sort_values(by=['Participant_ID', 'ticker', 'turn'])
merged_df_sorted['Close'] = merged_df_sorted.groupby(['Participant_ID', 'ticker'])['stockportfolio_after'].shift(0)
merged_df_sorted['Prev_Close'] = merged_df_sorted.groupby(['Participant_ID', 'ticker'])['stockportfolio_after'].shift(1)
merged_df_sorted['price_diff'] = merged_df_sorted['Close'] - merged_df_sorted['Prev_Close']

def get_trend_signal(diff):
    if pd.isna(diff): return 'neutral'
    if diff > 0: return 'buy'
    elif diff < 0: return 'sell'
    return 'neutral'
merged_df_sorted['graph_trend_signal'] = merged_df_sorted['price_diff'].apply(get_trend_signal)

def get_macd_signal(diff):
    if pd.isna(diff): return 'neutral'
    elif diff > 500: return 'buy'
    elif diff < -500: return 'sell'
    return 'neutral'
merged_df_sorted['macd_signal'] = merged_df_sorted['price_diff'].apply(get_macd_signal)

merged_df_sorted['rolling_std'] = merged_df_sorted.groupby(['Participant_ID', 'ticker'])['stockportfolio_after'].transform(lambda x: x.rolling(window=3, min_periods=2).std())
def get_bb_signal(row):
    std = row['rolling_std']
    diff = row['price_diff']
    if pd.isna(std) or std < 1000: return 'neutral'
    elif diff > 0: return 'buy'
    elif diff < 0: return 'sell'
    return 'neutral'
merged_df_sorted['bb_signal'] = merged_df_sorted.apply(get_bb_signal, axis=1)

# Merge with preference
merged = pd.merge(merged_df_sorted, preference_df, left_on='Participant_ID', right_on='Participant ID', how='left')
merged['action_clean'] = merged['action'].str.lower().str.strip()

def compute_match(merged, rank_col, signal_col, action_col='action_clean', label=''):
    df = merged[merged[rank_col] == 1].copy()
    df[f'{label}_match'] = df.apply(lambda row: 1 if row[action_col] == row[signal_col] else 0, axis=1)
    score = df.groupby('Participant_ID')[f'{label}_match'].agg(['mean', 'count']).reset_index()
    score.columns = ['Participant_ID', f'{label}_Rate', f'{label}_Turns']
    return score

graph_score = compute_match(merged, 'Graph_Rank', 'graph_trend_signal', label='Graph')
macd_score = compute_match(merged, 'MACD_Rank', 'macd_signal', label='MACD')
bb_score = compute_match(merged, 'BB_Rank', 'bb_signal', label='BB')

combined = graph_score.merge(macd_score, on='Participant_ID', how='outer')
combined = combined.merge(bb_score, on='Participant_ID', how='outer')

# Fill missing values
for col in combined.columns:
    if 'Rate' in col or 'Turns' in col:
        combined[col] = combined[col].fillna("0")

# Add Segments
def segment(row):
    segments = []
    if isinstance(row['Graph_Rate'], float) and row['Graph_Rate'] >= 0.7:
        segments.append('Consistent Graph User')
    elif isinstance(row['Graph_Rate'], float):
        segments.append('Inconsistent Graph User')
    if isinstance(row['MACD_Rate'], float) and row['MACD_Rate'] >= 0.7:
        segments.append('Consistent MACD User')
    elif isinstance(row['MACD_Rate'], float):
        segments.append('Inconsistent MACD User')
    if isinstance(row['BB_Rate'], float) and row['BB_Rate'] >= 0.7:
        segments.append('Consistent BB User')
    elif isinstance(row['BB_Rate'], float):
        segments.append('Inconsistent BB User')
    return ', '.join(segments) if segments else 'No Top-Ranked Indicator Evaluated'

combined['Segment'] = combined.apply(segment, axis=1)

# Export to your folder
os.makedirs(os.path.dirname(output_path), exist_ok=True)
combined.to_excel(output_path, index=False)
print(f"Exported successfully to: {output_path}")


Exported successfully to: E:\FYP\FYP Symposium\Output 2\Participant_Indicator_Segments.xlsx


In [17]:
#=====Block 9 Map The indicators to their preferences and score them. HL and OP and volatility should be weighted and avged out=====
import pandas as pd

# Path to your trend table Excel file
trend_file_path = r"E:\FYP\FYP Symposium\Outputs\Final_Trend_Tables.xlsx"

# Sheet names to load
sheet_names = ['TSLA', 'NFLX', 'PG', 'XOM']

# Create an empty DataFrame
indicator_map = pd.DataFrame()

# Loop through each sheet and combine data
for sheet in sheet_names:
    df = pd.read_excel(trend_file_path, sheet_name=sheet)
    df['Ticker'] = sheet  # Add ticker column
    indicator_map = pd.concat([indicator_map, df], ignore_index=True)

# --- Price + Volatility signal ---
def compute_price_vol_signal(row):
    trend = str(row.get('price_trend_7', '')).lower()
    vol = row.get('Volatility', None)

    price_score = 1 if trend == 'uptrend' else -1 if trend == 'downtrend' else 0

    if vol is None or pd.isna(vol):
        vol_score = 0
    elif vol < 10:
        vol_score = 1
    elif vol > 15:
        vol_score = -1
    else:
        vol_score = 0

    weighted_signal = 0.7 * price_score + 0.3 * vol_score
    return 'buy' if weighted_signal > 0 else 'sell'

indicator_map['PriceVol_Signal'] = indicator_map.apply(compute_price_vol_signal, axis=1)

# --- MACD + Histogram signal ---
def compute_macd_signal(row):
    macd_trend = str(row.get('MACD_trend_7', '')).lower()
    hist_strength = row.get('MACD Histogram Strength', 0)

    macd_score = 1 if macd_trend == 'buy' else -1 if macd_trend == 'sell' else 0
    hist_score = 1 if hist_strength > 0.5 else -1 if hist_strength < -0.5 else 0

    weighted_macd = 0.6 * macd_score + 0.4 * hist_score
    return 'buy' if weighted_macd > 0 else 'sell'

indicator_map['MACD_Signal'] = indicator_map.apply(compute_macd_signal, axis=1)

def score_bollinger(row):
    # Bollinger trend mapping
    bb = str(row['Bollinger Band Trend']).lower()
    bb_score = 1 if bb == 'oversold' else -1 if bb == 'overbought' else 0
    
    # Volatility normalized contribution (higher vol = less confidence)
    vol = row.get('Volatility', 0)
    vol_score = -1 if vol > 10 else (1 if vol < 5 else 0)

    # Weighted average
    final_score = (0.7 * bb_score) + (0.3 * vol_score)

    if final_score > 0.25:
        return 'Buy'
    elif final_score < -0.25:
        return 'Sell'
    else:
        return 'Neutral'

indicator_map['Bollinger_Signal'] = indicator_map.apply(score_bollinger, axis=1)

def score_volume(row):
    spike = row.get('Volume Spike', False)
    trend = str(row.get('volume_trend_7', '')).lower()

    if spike:
        if trend == 'uptrend':
            return 'Buy'
        elif trend == 'downtrend':
            return 'Sell'
        else:
            return 'Neutral'
    else:
        return 'Neutral'
        
indicator_map['Volume_Signal'] = indicator_map.apply(score_volume, axis=1)



# Preview key signals
print(indicator_map[['Turn', 'Ticker', 'PriceVol_Signal',
                    'MACD_Signal', 'Bollinger_Signal', 'Volume_Signal']].head(15))

# File path to save
export_path = r"E:\FYP\FYP Symposium\Output 2\Indicator_Map_With_Signals_CLEAN.xlsx"

# Columns to keep
final_cols = ['Turn', 'Ticker', 'PriceVol_Signal', 'MACD_Signal', 'Bollinger_Signal', 'Volume_Signal']

# Filter only required columns per ticker and save to separate sheets
with pd.ExcelWriter(export_path, engine='openpyxl') as writer:
    for ticker, df in indicator_map.groupby('Ticker'):
        df_final = df[final_cols].copy()
        df_final.to_excel(writer, sheet_name=ticker, index=False)

print(f"✅ Clean indicator signal file exported to: {export_path}")



#======Decision factors ko dimagh men rakh kar decision lena hai=====

import pandas as pd

# File path and sheet name
file_path = r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx"
sheet_name = 'Strategy'

# Columns to load
columns_to_use = [
    'Participant_ID',
    'Indicator_1st',
    'Indicator_2nd',
    'Indicator_3rd',
    'Indicator_4th',
    'Indicator_5th'
]

# Load only the specified columns
decision_factor_ranked = pd.read_excel(file_path, sheet_name=sheet_name, usecols=columns_to_use)

# Define the indicators we want to score
indicators = ['MACD', 'High/Low', 'Volume', 'Open/Close', 'Bollinger Bands']

# Initialize new columns with 0
for indicator in indicators:
    decision_factor_ranked[f'{indicator}_rank_score'] = 0

# Assign scores: 5 for 1st rank, down to 1 for 5th rank
score_map = {
    'Indicator_1st': 5,
    'Indicator_2nd': 4,
    'Indicator_3rd': 3,
    'Indicator_4th': 2,
    'Indicator_5th': 1
}

# Loop through rows and assign scores
for col, score in score_map.items():
    for indicator in indicators:
        decision_factor_ranked.loc[
            decision_factor_ranked[col] == indicator,
            f'{indicator}_rank_score'
        ] += score

# Calculate average score for price-based indicators: High/Low and Open/Close
decision_factor_ranked['price_rank_score'] = (
    decision_factor_ranked['High/Low_rank_score'] + decision_factor_ranked['Open/Close_rank_score']
) / 2

# Preview the result
print(decision_factor_ranked.head())

export_path = r"E:\FYP\FYP Symposium\Output 2\Decision_Factor_Ranked_Scores.xlsx"
decision_factor_ranked.to_excel(export_path, index=False)







    Turn Ticker PriceVol_Signal MACD_Signal Bollinger_Signal Volume_Signal
0      1   TSLA            sell        sell             Sell       Neutral
1      2   TSLA             buy        sell             Sell       Neutral
2      3   TSLA            sell        sell             Sell       Neutral
3      4   TSLA             buy        sell             Sell       Neutral
4      5   TSLA             buy        sell             Sell       Neutral
5      1   NFLX             buy        sell             Sell       Neutral
6      2   NFLX            sell        sell             Sell       Neutral
7      3   NFLX            sell        sell              Buy       Neutral
8      4   NFLX             buy        sell             Sell       Neutral
9      5   NFLX            sell        sell             Sell       Neutral
10     1     PG             buy         buy              Buy       Neutral
11     2     PG            sell         buy              Buy       Neutral
12     3     PG          

In [18]:
#==== Block 10: Load merged log and strategy sheet to see indicator and preferences====#

import pandas as pd

# Load all sheets from the Indicator Map file
indicator_file = r"E:\FYP\FYP Symposium\Output 2\Indicator_Map_With_Signals_CLEAN.xlsx"
all_sheets = pd.read_excel(indicator_file, sheet_name=None)

# Combine all sheets into one DataFrame
indicator_map = pd.concat(
    [df.assign(Ticker=name) for name, df in all_sheets.items()],
    ignore_index=True
)

# Ensure consistent column naming
indicator_map.rename(columns={'Turn': 'turn'}, inplace=True)

# Load the base participant data
log_file = r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx"
participant_cols = [
    'Date', 'Real Time', 'Simulation Time', 'Seconds left',
    'Participant_ID', 'turn', 'action', 'ticker'
]
participant_df = pd.read_excel(log_file, sheet_name='Merged_Log', usecols=participant_cols)

# Merge signal info from indicator_map using both 'turn' and 'ticker'
merged_df = participant_df.merge(
    indicator_map[['turn', 'Ticker', 'PriceVol_Signal', 'MACD_Signal', 'Bollinger_Signal', 'Volume_Signal']],
    how='left',
    left_on=['turn', 'ticker'],
    right_on=['turn', 'Ticker']
)

# Drop the duplicate 'Ticker' column
merged_df.drop(columns='Ticker', inplace=True)

# Load decision factor scores
ranked_file = r"E:\FYP\FYP Symposium\Output 2\Decision_Factor_Ranked_Scores.xlsx"
ranked_df = pd.read_excel(ranked_file)

# Drop High/Low and Open/Close rank scores
ranked_df = ranked_df.drop(columns=['High/Low_rank_score', 'Open/Close_rank_score'])

# Merge scores based on Participant_ID
final_df = merged_df.merge(ranked_df, on='Participant_ID', how='left')

# === Assign action-based signal scores ===
def compute_score(row, signal_col, score_col):
    signal = str(row[signal_col]).strip().lower()
    action = str(row['action']).strip().lower()
    rank_score = row.get(score_col, 0)

    if signal == 'neutral':
        return rank_score / 2
    elif signal == action:
        return rank_score
    else:
        return 0

# Apply the logic to each indicator
final_df['PriceVol_action_score'] = final_df.apply(lambda r: compute_score(r, 'PriceVol_Signal', 'High/Low_rank_score'), axis=1)
final_df['MACD_action_score'] = final_df.apply(lambda r: compute_score(r, 'MACD_Signal', 'MACD_rank_score'), axis=1)
final_df['Bollinger_action_score'] = final_df.apply(lambda r: compute_score(r, 'Bollinger_Signal', 'Bollinger Bands_rank_score'), axis=1)
final_df['Volume_action_score'] = final_df.apply(lambda r: compute_score(r, 'Volume_Signal', 'Volume_rank_score'), axis=1)

# Add Total Action Score by summing all individual action scores
final_df['Total_action_score'] = (
    final_df['PriceVol_action_score'] +
    final_df['MACD_action_score'] +
    final_df['Bollinger_action_score'] +
    final_df['Volume_action_score']
)

final_df = final_df[final_df['turn'] != 6]

# Add explanation for Total_action_score
def explain_action_alignment(row):
    scores = {
        'PriceVol': row['PriceVol_action_score'],
        'MACD': row['MACD_action_score'],
        'Bollinger': row['Bollinger_action_score'],
        'Volume': row['Volume_action_score']
    }
    strong_matches = [k for k, v in scores.items() if v >= 0.75]
    neutral_matches = [k for k, v in scores.items() if 0 < v < 0.75]
    misses = [k for k, v in scores.items() if v == 0]

    explanation = []
    if strong_matches:
        explanation.append(f"Strong alignment with {', '.join(strong_matches)}")
    if neutral_matches:
        explanation.append(f"Partially followed signals from {', '.join(neutral_matches)}")
    if misses:
        explanation.append(f"Ignored or mismatched {', '.join(misses)}")

    return " | ".join(explanation)

final_df['Action_Score_Explanation'] = final_df.apply(explain_action_alignment, axis=1)

#Pulled the three columns from BLOCK 3 about knowledge from pre data if they know how to use what indicator

def check_knowledge(prm2b_14a_response):
    if pd.isna(prm2b_14a_response):
        return {'MACD': False, 'Bollinger': False, 'Volume': False, 'HighLow_OpenClose': False}

    text = prm2b_14a_response.lower()
    
    knowledge = {
        'MACD': False,
        'Bollinger': False,
        'Volume': False,
        'HighLow_OpenClose': False
    }

    # Check MACD knowledge
    if 'macd' in text:
        knowledge['MACD'] = True
    
    # Check Bollinger Bands
    if 'bollinger' in text or 'bb' in text:
        knowledge['Bollinger'] = True

    # Check Volume
    if 'volume' in text:
        knowledge['Volume'] = True

    # Check High Low Open Close
    if 'high' in text or 'low' in text or 'open' in text or 'close' in text or 'ohlc' in text:
        knowledge['HighLow_OpenClose'] = True

    return knowledge

# --- Apply to all participants ---

knowledge_records = []

for idx, row in strategy.iterrows():
    participant_id = row['Participant_ID']
    prm2b_14a = row.get('PRM2b_14a', None)
    knowledge = check_knowledge(prm2b_14a)

    knowledge_records.append({
        'Participant_ID': participant_id,
        'Knows_MACD': 'Yes' if knowledge['MACD'] else 'No',
        'Knows_Bollinger': 'Yes' if knowledge['Bollinger'] else 'No',
        'Knows_Volume': 'Yes' if knowledge['Volume'] else 'No',
        'Knows_HighLow_OpenClose': 'Yes' if knowledge['HighLow_OpenClose'] else 'No'
    })

# Create DataFrame
knowledge_df = pd.DataFrame(knowledge_records)

# Drop 'Knows_HighLow_OpenClose' — no longer needed
if 'Knows_HighLow_OpenClose' in knowledge_df.columns:
    knowledge_df.drop(columns=['Knows_HighLow_OpenClose'], inplace=True)

# --- Merge Cleaned Knowledge into Strategy ---

# Drop old versions to avoid MergeError
strategy.drop(columns=['Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knows_HighLow_OpenClose', 'Knowledge_Score'], errors='ignore', inplace=True)

# Merge new knowledge
strategy = pd.merge(strategy, knowledge_df, on='Participant_ID', how='left')
print("\n✅ Merged updated knowledge into strategy successfully.")

# --- Scoring Logic ---

# Map Yes/No with new scoring logic (Volume: Yes = 1, No = 0.5)
knowledge_df_numeric = knowledge_df.copy()
knowledge_df_numeric['Knows_MACD'] = knowledge_df_numeric['Knows_MACD'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Bollinger'] = knowledge_df_numeric['Knows_Bollinger'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Volume'] = knowledge_df_numeric['Knows_Volume'].map({'Yes': 1, 'No': 0.5})

# Compute score
knowledge_df['Knowledge_Score'] = (
    knowledge_df_numeric['Knows_MACD'] +
    knowledge_df_numeric['Knows_Bollinger'] +
    knowledge_df_numeric['Knows_Volume']
)

# Show sample output
print("\n✅ Final Knowledge Score (range 0.5–3.0):")
display(knowledge_df[['Participant_ID', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knowledge_Score']].head())

# Optional documentation string
knowledge_score_note = "Knowledge_Score = Knows_MACD (1/0) + Knows_Bollinger (1/0) + Knows_Volume (1 if Yes, 0.5 if No); Range = 0.5 to 3.0"


final_df = final_df.merge(strategy[['Participant_ID', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume']], on='Participant_ID', how='left')

def compute_used_know_score(row, signal_col, know_col):
    signal = str(row[signal_col]).strip().lower()
    action = str(row['action']).strip().lower()
    knows = row[know_col]

    if signal == action and knows == 'Yes':
        return 1
    else:
        return 0

# Apply logic for each indicator
final_df['used_MACD_Know_Score'] = final_df.apply(lambda r: compute_used_know_score(r, 'MACD_Signal', 'Knows_MACD'), axis=1)
final_df['used_Bollinger_Know_Score'] = final_df.apply(lambda r: compute_used_know_score(r, 'Bollinger_Signal', 'Knows_Bollinger'), axis=1)
final_df['used_Volume_Know_Score'] = final_df.apply(lambda r: compute_used_know_score(r, 'Volume_Signal', 'Knows_Volume'), axis=1)

# Optional: total informed usage score
final_df['Total_Used_Know_Score'] = (
    final_df['used_MACD_Know_Score'] +
    final_df['used_Bollinger_Know_Score'] +
    final_df['used_Volume_Know_Score']
)

final_df['Combined_Action_Knowledge_Score'] = (
    final_df['Total_action_score'] + final_df['Total_Used_Know_Score']
)


print(final_df[['Participant_ID', 'turn', 'ticker', 'action',
                'PriceVol_action_score', 'MACD_action_score',
                'Bollinger_action_score', 'Volume_action_score',
                'Total_action_score', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume',
                'used_MACD_Know_Score', 'used_Bollinger_Know_Score', 'used_Volume_Know_Score', 'Combined_Action_Knowledge_Score']].head(2))

# Export the final dataframe
final_df.to_excel(
    r"E:\FYP\FYP Symposium\Output 2\participant_indicator_preference_with_signals.xlsx",
    index=False
)

print(f"✅ Final participant-indicator preference file with action scores exported to: {export_path}")



✅ Merged updated knowledge into strategy successfully.

✅ Final Knowledge Score (range 0.5–3.0):


,Participant_ID,Knows_MACD,Knows_Bollinger,Knows_Volume,Knowledge_Score
0,E0070,No,No,No,0.5
1,E0169,Yes,Yes,Yes,3.0
2,E0426,No,No,No,0.5
3,E0712,No,No,No,0.5
4,E1130,Yes,No,No,1.5


  Participant_ID  turn ticker action  PriceVol_action_score  \
0          E0070     1   TSLA    Buy                      0   
1          E0070     2   TSLA   Sell                      0   

   MACD_action_score  Bollinger_action_score  Volume_action_score  \
0                  0                       0                  2.0   
1                  1                       2                  2.0   

   Total_action_score Knows_MACD Knows_Bollinger Knows_Volume  \
0                 2.0         No              No           No   
1                 5.0         No              No           No   

   used_MACD_Know_Score  used_Bollinger_Know_Score  used_Volume_Know_Score  \
0                     0                          0                       0   
1                     0                          0                       0   

   Combined_Action_Knowledge_Score  
0                              2.0  
1                              5.0  
✅ Final participant-indicator preference file with action sc

In [19]:
########################## # BLOCK 11: Assign Rationality (Final Version)
# --- Merge Extracted Knowledge into Strategy ---

# Assuming 'knowledge_df' already created
# Merge knowledge_df with strategy on 'Participant_ID'
# Drop old columns if exist to avoid suffix errors
cols_to_drop = ['Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knows_HighLow_OpenClose']
strategy = strategy.drop(columns=[col for col in cols_to_drop if col in strategy.columns], errors='ignore')

# Now merge

strategy = pd.merge(strategy, knowledge_df, on='Participant_ID', how='left')

print("\n✅ Merged Participant Knowledge into Strategy Frame.")

def assign_rationality(row):
    participant_id = row['Participant_ID']
    action = str(row['action']).lower()
    ticker = str(row['ticker']).upper()
    turn = row['turn']

    participant_row = strategy[strategy['Participant_ID'] == participant_id]
    if participant_row.empty:
        return pd.Series(['Unknown', np.nan, np.nan, np.nan, np.nan])

    decision_factors = [
        (participant_row['DecisionFactor_1st'].values[0], 0.5),
        (participant_row['DecisionFactor_2nd'].values[0], 0.3),
        (participant_row['DecisionFactor_3rd'].values[0], 0.2)
    ]
    indicators = [
        (participant_row['Indicator_1st'].values[0], 0.5),
        (participant_row['Indicator_2nd'].values[0], 0.4),
        (participant_row['Indicator_3rd'].values[0], 0.3),
        (participant_row['Indicator_4th'].values[0], 0.2),
        (participant_row['Indicator_5th'].values[0], 0.1)
    ]
    known_indicators = participant_row['PRM2b_14a'].values[0]

    trend_table = trend_tables.get(ticker)
    if trend_table is None:
        return pd.Series(['Unknown', np.nan, np.nan, np.nan, np.nan])

    trend_row = trend_table[trend_table['Turn'] == turn]
    if trend_row.empty:
        return pd.Series(['Unknown', np.nan, np.nan, np.nan, np.nan])

    trend_row = trend_row.iloc[0]

    # Step 1: Emotionally Driven
    if str(row['news_truth']).lower() == 'false':
        if (str(row['news_sentiment']).lower() == 'positive' and action == 'buy') or \
           (str(row['news_sentiment']).lower() == 'negative' and action == 'sell'):
            return pd.Series(['Emotionally Driven', 0.0, 0.0, 0.0, 0.0])

    # Step 2: Rationality Scoring
    rationality_score = 0
    breakdown_scores = {
        'DecisionFactor_Score': 0,
        'Indicator_Score': 0,
        'Knowledge_Bonus': 0
    }

    for factor, weight in decision_factors:
        if str(factor).lower() == 'graph':
            if (trend_row['Price Trend'] == 'uptrend' and action == 'buy') or \
               (trend_row['Price Trend'] == 'downtrend' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['DecisionFactor_Score'] += weight
        elif str(factor).lower() == 'news':
            if (row['news_sentiment'].lower() == 'positive' and row['news_truth'].lower() == 'true' and action == 'buy') or \
               (row['news_sentiment'].lower() == 'negative' and row['news_truth'].lower() == 'true' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['DecisionFactor_Score'] += weight

    for indicator, weight in indicators:
        if pd.isna(indicator):
            continue
        if str(indicator).lower() == 'volume':
            if trend_row['Volume Spike'] and action == 'buy':
                rationality_score += weight
                breakdown_scores['Indicator_Score'] += weight
                if participant_row['Knows_Volume'].values[0] == 'Yes':
                    rationality_score += 0.2
                    breakdown_scores['Knowledge_Bonus'] += 0.2
        elif str(indicator).lower() == 'macd':
            if (trend_row['MACD Signal'] == 'buy' and action == 'buy') or (trend_row['MACD Signal'] == 'sell' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['Indicator_Score'] += weight
                if participant_row['Knows_MACD'].values[0] == 'Yes':
                    rationality_score += 0.2
                    breakdown_scores['Knowledge_Bonus'] += 0.2
        elif str(indicator).lower() == 'bollinger bands':
            if (trend_row['Bollinger Band Trend'] == 'oversold' and action == 'buy') or \
               (trend_row['Bollinger Band Trend'] == 'overbought' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['Indicator_Score'] += weight
                if participant_row['Knows_Bollinger'].values[0] == 'Yes':
                    rationality_score += 0.2
                    breakdown_scores['Knowledge_Bonus'] += 0.2

    # Step 3: Label
    if rationality_score >= 0.7:
        label = 'Fully Rational'
    elif 0.4 <= rationality_score < 0.7:
        label = 'Slightly Rational'
    else:
        label = 'Irrational'

    return pd.Series([
        label,
        rationality_score,
        breakdown_scores['DecisionFactor_Score'],
        breakdown_scores['Indicator_Score'],
        breakdown_scores['Knowledge_Bonus']
    ])
merged_log[['rationality_label', 'rationality_score', 'DecisionFactor_Score', 'Indicator_Score', 'Knowledge_Bonus']] = merged_log.apply(assign_rationality, axis=1)


print("\n✅ Rationality Label and Score Assigned to merged_log.")

from openpyxl import load_workbook
from datetime import datetime

# Generate Timestamp Name
current_time = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
output_path = rf'E:\FYP\FYP Symposium\Outputs\Behavioral_Rationality_Log_{current_time}.xlsx'

# Save merged_log
merged_log.to_excel(output_path, index=False)

print(f"\n✅ Behavioral Rationality Log Saved Successfully at {output_path}")

# Now prepare Market Signals + Rationality
# Now prepare Market Signals + Rationality
# Prepare Market Signals + Rationality
market_signal_records = []

for idx, row in merged_log.iterrows():
    ticker = str(row['ticker']).upper()
    turn = row['turn']
    participant_id = row['Participant_ID']
    rationality = row['rationality_label']
    rationality_score = row['rationality_score']

    trend_table = trend_tables.get(ticker)
    if trend_table is None:
        continue

    trend_row = trend_table[trend_table['Turn'] == turn]
    if trend_row.empty:
        continue

    trend_row = trend_row.iloc[0]
    market_signal_records.append({
        'Participant_ID': participant_id,
        'Turn': turn,
        'Ticker': ticker,
        'Volatility': trend_row['Volatility'],
        'Price Trend': trend_row['Price Trend'],
        'MACD Signal': trend_row['MACD Signal'],
        'MACD Histogram': trend_row['MACD Histogram'],
        'Volume Spike': trend_row['Volume Spike'],
        'Bollinger Band Trend': trend_row['Bollinger Band Trend'],
        'Is Attractive to Buy?': trend_row['Is Attractive to Buy?'],
        'DecisionFactor_Score': row['DecisionFactor_Score'],
        'Indicator_Score': row['Indicator_Score'],
        'Knowledge_Bonus': row['Knowledge_Bonus'],
        'Rationality Score': rationality_score,
        'Rationality Label': rationality
    })


market_signals_df = pd.DataFrame(market_signal_records)
# Now open in append mode to add new sheet
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
    market_signals_df.to_excel(writer, sheet_name='MarketSignals_WithRationality', index=False)

print("\n✅ Market Signals + Rationality Sheet Saved Successfully!")

# --- Create Participant Behavioral Summary ---

# --- Create Full Participant Behavioral Summary ---

summary_records = []

participants = merged_log['Participant_ID'].unique()

for pid in participants:
    participant_data = merged_log[merged_log['Participant_ID'] == pid]
    total_decisions = len(participant_data)

    fully_rational = len(participant_data[participant_data['rationality_label'] == 'Fully Rational'])
    slightly_rational = len(participant_data[participant_data['rationality_label'] == 'Slightly Rational'])
    irrational = len(participant_data[participant_data['rationality_label'] == 'Irrational'])
    emotionally_driven = len(participant_data[participant_data['rationality_label'] == 'Emotionally Driven'])

    summary_records.append({
        'Participant_ID': pid,
        'Total Actions': total_decisions,
        '# Fully Rational': fully_rational,
        '# Slightly Rational': slightly_rational,
        '# Irrational': irrational,
        '# Emotionally Driven': emotionally_driven,
        '% Fully Rational': round((fully_rational / total_decisions) * 100, 2),
        '% Slightly Rational': round((slightly_rational / total_decisions) * 100, 2),
        '% Irrational': round((irrational / total_decisions) * 100, 2),
        '% Emotionally Driven': round((emotionally_driven / total_decisions) * 100, 2),
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_records)

# Determine Overall Participant Classification
def classify_participant(row):
    labels = {
        'Fully Rational': row['% Fully Rational'],
        'Slightly Rational': row['% Slightly Rational'],
        'Irrational': row['% Irrational'],
        'Emotionally Driven': row['% Emotionally Driven']
    }
    dominant = max(labels, key=labels.get)
    return dominant

summary_df['Overall Classification'] = summary_df.apply(classify_participant, axis=1)

# Save to the Excel file in a new sheet
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
    summary_df.to_excel(writer, sheet_name='Participant_Summary', index=False)

print("\n✅ Participant Behavioral Summary Sheet Saved Successfully!")
print("\n📊 FINAL OUTPUT: Top 10 Rows of Market Signals + Rationality Data:\n")
print(market_signals_df.head(10).to_string(index=False))


✅ Merged Participant Knowledge into Strategy Frame.

✅ Rationality Label and Score Assigned to merged_log.

✅ Behavioral Rationality Log Saved Successfully at E:\FYP\FYP Symposium\Outputs\Behavioral_Rationality_Log_2025-05-02 05-25-59.xlsx

✅ Market Signals + Rationality Sheet Saved Successfully!

✅ Participant Behavioral Summary Sheet Saved Successfully!

📊 FINAL OUTPUT: Top 10 Rows of Market Signals + Rationality Data:

Participant_ID  Turn Ticker  Volatility Price Trend MACD Signal  MACD Histogram  Volume Spike Bollinger Band Trend Is Attractive to Buy?  DecisionFactor_Score  Indicator_Score  Knowledge_Bonus  Rationality Score  Rationality Label
         E0070     1   TSLA   26.239602     neutral     neutral           -7.68         False              neutral                 Risky                   0.0              0.0              0.0                0.0         Irrational
         E0070     2   TSLA   11.078977     neutral     neutral           -2.91         False              neut

In [20]:
##########################
# BLOCK 12: Compute Final Decision Quality Score (Rationality Proxy)
##########################

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load files
pre_survey = pd.read_excel(r"E:\FYP\FYP Symposium\Pre-Survey Form (Responses).xlsx")
post_survey = pd.read_excel(r"E:\FYP\FYP Symposium\Post-Survey (Responses).xlsx")
merged_log = pd.read_excel(r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx")

# Clean column headers
pre_survey.columns = pre_survey.columns.str.strip()
post_survey.columns = post_survey.columns.str.strip()
merged_log.columns = merged_log.columns.str.strip()

# Rename pre-survey columns
pre_survey = pre_survey.rename(columns={
    'Participant ID (AXXXX format)\nIf ID is not assigned yet, kindly contact researchers before proceeding': 'Participant_ID',
    'How confident are you in your ability to make profitable trades?': 'Confidence',
    'When faced with Financial decisions, do you usually rely on Logic or Intuition?': 'Logic_vs_Intuition',
    'How comfortable are you with taking risks?': 'Risk_Tolerance',
    'How comfortable are you with uncertainty in Financial Markets?': 'Comfort_with_Uncertainty',
    'Do you consider yourself patient or impulsive when making decisions?': 'Impulsiveness'
})

# Select pre-survey features
pre_features = pre_survey[['Participant_ID', 'Confidence', 'Logic_vs_Intuition', 'Risk_Tolerance',
                           'Comfort_with_Uncertainty', 'Impulsiveness']]

# Rename and select post-survey columns
post_survey = post_survey.rename(columns={
    'Participant ID': 'Participant_ID',
    'How much pressure did you feel during the experiment to perform well?': 'Pressure',
    'How well do you think you have performed in the trading simulation?': 'Self_Performance',
    'How well do you think you may have performed as compared to other participants?': 'Relative_Performance'
})
post_features = post_survey[['Participant_ID', 'Pressure', 'Self_Performance', 'Relative_Performance']]

# Behavioral summary
log_summary = merged_log.groupby('Participant_ID').agg(
    Total_Trades=('action', 'count'),
    Unique_Tickers=('ticker', pd.Series.nunique)
).reset_index()

# Final cash (profit proxy) — last row per participant
profit_summary = merged_log.sort_values(by=['Participant_ID', 'turn']).groupby('Participant_ID').tail(1)[
    ['Participant_ID', 'cash_after']
].rename(columns={'cash_after': 'Final_Cash'})

# Merge all features
merged_df = pre_features.merge(post_features, on='Participant_ID', how='inner')
merged_df = merged_df.merge(log_summary, on='Participant_ID', how='inner')
merged_df = merged_df.merge(profit_summary, on='Participant_ID', how='left')

# Log-transform cash
merged_df['Log_Cash'] = np.log(merged_df['Final_Cash'])

# Normalize using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_df.drop(columns=['Participant_ID', 'Final_Cash']))
scaled_df = pd.DataFrame(scaled_data, columns=merged_df.columns.drop(['Participant_ID', 'Final_Cash']))
scaled_df['Participant_ID'] = merged_df['Participant_ID']
scaled_df['Final_Cash'] = merged_df['Final_Cash']  # Keep original value

# Compute Final Decision Quality Score
scaled_df['Final_Decision_Quality_Score'] = (
    0.25 * scaled_df[['Confidence', 'Logic_vs_Intuition', 'Risk_Tolerance',
                      'Comfort_with_Uncertainty', 'Impulsiveness']].mean(axis=1) +
    0.20 * scaled_df[['Pressure', 'Self_Performance', 'Relative_Performance']].mean(axis=1) +
    0.15 * scaled_df[['Total_Trades', 'Unique_Tickers']].mean(axis=1) +
    0.40 * scaled_df['Log_Cash']
)

# ================================
# Example Categorization: Rationality Label
# ================================
scaled_df['Rationality_Label'] = pd.cut(
    scaled_df['Final_Decision_Quality_Score'],
    bins=[0, 0.4, 0.7, 1.0],
    labels=['Irrational', 'Moderate', 'Rational']
)
# Add Explanation based on Rationality Label and Score
def explain_rationality(score, label):
    if score >= 0.85:
        return "Highly rational; confident, logical, and performed strongly under pressure with high profit."
    elif score >= 0.70:
        return "Generally rational; showed strong strategy use and psychological stability."
    elif score >= 0.55:
        return "Moderately effective; balanced decision-making but either cautious or inconsistent performance."
    elif score >= 0.40:
        return "Somewhat impulsive or stressed; moderate trading performance and mixed self-perception."
    else:
        return "Likely struggled under pressure; low confidence, impulsive patterns, or poor final performance."

scaled_df['Rationality_Explanation'] = scaled_df.apply(
    lambda row: explain_rationality(row['Final_Decision_Quality_Score'], row['Rationality_Label']),
    axis=1
)

# Add breakdown explanation per participant
def build_score_formula(row):
    cog = round(row[['Confidence', 'Logic_vs_Intuition', 'Risk_Tolerance',
                     'Comfort_with_Uncertainty', 'Impulsiveness']].mean(), 2)
    perc = round(row[['Pressure', 'Self_Performance', 'Relative_Performance']].mean(), 2)
    behav = round(row[['Total_Trades', 'Unique_Tickers']].mean(), 2)
    perf = round(row['Log_Cash'], 2)

    return f"Score = 25% Cognitive ({cog}) + 20% Perception ({perc}) + 15% Behavior ({behav}) + 40% Log-Cash ({perf})"

scaled_df['Score_Breakdown'] = scaled_df.apply(build_score_formula, axis=1)


# Save results
scaled_df[['Participant_ID', 'Final_Decision_Quality_Score', 'Rationality_Label', 'Rationality_Explanation', 'Score_Breakdown']].to_excel(
    r"E:\FYP\FYP Symposium\Output 2\Final_Decision_Quality_Scores.xlsx", index=False)

print("✅ Final Decision Quality Scores (with final cash) computed and saved.")


✅ Final Decision Quality Scores (with final cash) computed and saved.
